# Shark!!!!!
## Data driven conclusions on shark attacks worldwide.


The data that has no interest must first be removed so it is easier to draw a hypothesis.

In [1]:
import pandas as pd
import numpy as np

In [2]:
rawData = pd.read_csv('GSAF5.csv', encoding = 'ISO-8859-1') # reading de csv file using pandas required using special encoding
rawData.head() #shows firs 5 rows of data so we can determine disposable data

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


### Droping columns
Several columns can be droped from the data since they don't provide usefull information to reach any conclusions.

In [3]:
data = rawData.drop(['Case Number', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23'], axis = 1)
data.head()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016"
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016"
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016"
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016"
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016"


With the new "data" table it's easier to analize the usable information and decide a hypothesis that can be tested with the available data.

## Hypothesis:

Given the available data, it is interesting to see if there is a relation between tha fatlity of the attacks and the activity that the victim was doing.  A conclusion could be reached identifying the higher risk activities related to shark attacks.

## Data Cleaning

In an initial visual inspection of the head of the data table it is evident that "Age" & "Species" columns are missing data, so further analisis searching for missing values is in order.

In [4]:
null_cols = data.isnull().sum()
print(null_cols[null_cols > 0])
print('Data entrys: {}'.format(len(data.index)-1))

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
dtype: int64
Data entrys: 5991


"Age", "Time" and "Species" have a significant ammount of null data.  These columns can be deleted since over half the entrys are empty.

A limit of 1000 empty cells is selected.  If the data  column has more than 1000 empty cells (17%) it is deleted.

From observing the first rows of the remaining data, we can see that the "Name" column has currupt data (male as name), and since the individual name of the victim is of no interest to us, we can delete the column.

The column "Injury" has a very specific discription of the victim's injury.  This type of data is hard to cuantify and the reult of the injury can be found on the "Fatal" column, so we can delete this column too.

The "Investigator or Source" column can be deleted since we have decided to trust all the data from the data set ignoring its origin.

In [5]:
drop_cols = list(null_cols[null_cols > 1000].index)
data = data.drop(drop_cols, axis=1)
data = data.drop(['Name' ,'Injury', 'Investigator or Source'], axis = 1)

data.head()

,Date,Year,Type,Country,Area,Location,Activity,Sex,Fatal (Y/N)
0,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N
1,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N
2,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N
3,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,N
4,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,N


The core focus of the investigation is ACTIVITY, so a deeper analysis is in order.

In [6]:
data['Activity'].fillna('Unknown', inplace = True) #replace all NaN values in Activity to "Unknown"
data['Fatal (Y/N)'].fillna('Unknown', inplace = True)

In [7]:
data.Activity.value_counts()

Surfing                                                                                                                                                                                                                                     904
Swimming                                                                                                                                                                                                                                    819
Unknown                                                                                                                                                                                                                                     530
Fishing                                                                                                                                                                                                                                     414
Spearfishing                            